In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import numpy as np
import copy

data_dir = "/kaggle/input/AI-OF-GOD-4/aog_data/train"
num_classes = 9
batch_size = 16
num_epochs = 10
num_folds = 5
lr = 1e-4
start_fold = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)
targets = [label for _, label in full_dataset.imgs]
targets = np.array(targets)

skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(targets)), targets)):
    if fold < start_fold:
        print(f"\n✅ Skipping Fold {fold + 1} (already trained)\n")
        continue

    print(f"\n======================== Fold {fold + 1}/{num_folds} ========================\n")

    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(copy.deepcopy(full_dataset), val_idx)
    val_subset.dataset.transform = val_transform

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=2)

    model = models.maxvit_t(weights=models.MaxVit_T_Weights.IMAGENET1K_V1)
    model.classifier[5] = nn.Linear(model.classifier[5].in_features, num_classes)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 40)

        model.train()
        train_loss, train_preds, train_labels = 0.0, [], []
        for imgs, labels in tqdm(train_loader, desc="Training"):
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * imgs.size(0)
            train_preds.extend(outputs.argmax(1).cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_acc = accuracy_score(train_labels, train_preds)
        train_loss /= len(train_subset)

        model.eval()
        val_loss, val_preds, val_labels = 0.0, [], []
        with torch.no_grad():
            for imgs, labels in tqdm(val_loader, desc="Validation"):
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * imgs.size(0)
                val_preds.extend(outputs.argmax(1).cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_acc = accuracy_score(val_labels, val_preds)
        val_loss /= len(val_subset)
        scheduler.step()

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            save_path = f"best_maxvit_fold_{fold+1}.pth"
            torch.save(model.state_dict(), save_path)
            print(f"✅ Saved Best Model for Fold {fold+1} at {save_path}")

    print(f"\n✅ Fold {fold+1} Best Val Accuracy: {best_acc:.4f}\n")

print("🔥 Training completed successfully on single GPU!")

✅ Using device: cuda

✅ Skipping Fold 1 (already trained)


======================== Fold 2/5 ========================

Epoch 1/10
----------------------------------------


Validation: 100%|██████████| 154/154 [00:20<00:00,  7.52it/s]


Train Loss: 0.6116 | Train Acc: 0.8048
Val Loss:   0.2676 | Val Acc:   0.9147
✅ Saved Best Model for Fold 2 at best_maxvit_fold_2.pth
Epoch 2/10
----------------------------------------


Validation: 100%|██████████| 154/154 [00:20<00:00,  7.55it/s]


Train Loss: 0.2313 | Train Acc: 0.9280
Val Loss:   0.2813 | Val Acc:   0.9041
Epoch 3/10
----------------------------------------


...

Validation: 100%|██████████| 154/154 [00:20<00:00,  7.55it/s]


Train Loss: 0.6344 | Train Acc: 0.7980
Val Loss:   0.3184 | Val Acc:   0.8960
✅ Saved Best Model for Fold 3 at best_maxvit_fold_3.pth
Epoch 2/10
----------------------------------------


Validation: 100%|██████████| 154/154 [00:20<00:00,  7.55it/s]


Train Loss: 0.2365 | Train Acc: 0.9229
Val Loss:   0.2013 | Val Acc:   0.9347
✅ Saved Best Model for Fold 3 at best_maxvit_fold_3.pth
Epoch 3/10
----------------------------------------


Training:  44%|████▍     | 270/613 [01:49<02:19,  2.46it/s]

In [ ]:
print("ello")

In [ ]:
print("Hello")

In [ ]:
print("Hiiii")

In [ ]:
print("HELLO!")

In [ ]:
print("Hello!")

In [ ]:
print("Hello!")